In [ ]:
# Install and configure ngrok
!apt install curl
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | tee /etc/apt/sources.list.d/ngrok.list && apt update && apt install ngrok
!ngrok config add-authtoken 2GHKAmPiTsA9pPwnAPKxVYIl8iE_mKCeBCh8h8qoaoCEAjjw

In [ ]:
# Copy the flask repository
!git clone https://github.com/r0zh/Visionhub-flask-models

In [ ]:
cd Visionhub-flask-models

In [ ]:
# Install the requirements
!pip install -r requirements.txt

In [ ]:
# Copy the shap-e repository
!git clone https://github.com/openai/shap-e.git

In [ ]:
# Install the shap-e package
!pip install -e ./shap-e

In [ ]:
cd shap-e/

In [6]:
import torch
import trimesh

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

from pyngrok import ngrok

In [7]:
# Try to use the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
# Load the models
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [ ]:
# FLASK SERVER
from flask import Flask, send_from_directory, request
from shap_e.util.notebooks import decode_latent_mesh

# Initialize the Flask application
app = Flask(__name__)
port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

@app.route('/get_model', methods=['POST'])
def get_photo():
    guidance_scale = 3.0

    # To get the best result, you should remove the background and show only the object of interest to the model.
    prompt = request.get_json()['prompt']
    if(request.get_json()['batch_size']):
      batch_size = request.get_json()['batch_size']
    else:
      batch_size = 1
    print(prompt)


    latents = sample_latents(
      batch_size=batch_size,
      model=model,
      diffusion=diffusion,
      guidance_scale=guidance_scale,
      model_kwargs=dict(texts=[prompt] * batch_size),
      progress=True,
      clip_denoised=True,
      use_fp16=True,
      use_karras=True,
      karras_steps=64,
      sigma_min=1e-3,
      sigma_max=160,
      s_churn=0,
    )

    for i, latent in enumerate(latents):
      t = decode_latent_mesh(xm, latent).tri_mesh()
      with open(f'example_mesh_{i}.ply', 'wb') as f:
          t.write_ply(f)

      with open(f'example_mesh_{i}.obj', 'w') as f:
          t.write_obj(f)
          mesh = trimesh.load(f'example_mesh_{i}.obj')
          mesh.export('output.glb', file_type='glb')


    return send_from_directory("/content/Visionhub-flask-models/shap-e", "example_mesh_0.ply")

# Start the Flask server
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

In [ ]:
from PIL import Image
import PIL

render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)

    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)
        mesh = trimesh.load(f'example_mesh_{i}.obj')
        mesh.export('output.glb', file_type='glb')



In [ ]:
def make_gif(frame_folder):
    frames = frame_folder
    frame_one = frames[0]
    frame_one.save("download.gif", format="GIF", append_images=frames,
                   save_all=True, duration=1000, loop=1)